In [1]:
import copy
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics
from sklearn import datasets
import matplotlib.pyplot as plt
from xgboost import plot_importance
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc, roc_auc_score

In [2]:
data = pd.read_csv('train.csv')
quiz = pd.read_csv('test_data.csv')

In [3]:

drop_out = [data.columns[0]]
for i in data.columns[5:]:
    if 1-len(data[data[i].notna()])/len(data)>0.3:
        drop_out.append(i)
print(drop_out)
data = data.drop(columns=drop_out)
quiz = quiz.drop(columns=drop_out)
data

['Unnamed: 0', 'COL13', 'COL14', 'COL21', 'COL25', 'COL31', 'COL37', 'COL45', 'COL46', 'COL52']


ID  CHANNEL_A  CHANNEL_B  CHANNEL_C   COL1  COL2 COL3 COL4 COL5  \
0      T000689          0          0          0    NaN    58    A    B    E   
1      T000848          0          1          0    6.0    49    B    B    C   
2      T000967          0          0          0    NaN    55    B    A    C   
3      T001249          0          0          0   17.0    36    K    A    B   
4      T001524          0          1          0   22.0    43  NaN  NaN    B   
...        ...        ...        ...        ...    ...   ...  ...  ...  ...   
99995  T034492          0          1          0    NaN    30    K    A    A   
99996  T034682          0          0          0    4.0    49    Z    B    B   
99997  T035207          0          0          0   18.0    30    B    A    A   
99998  T035311          0          0          0  347.0    33    B    B    A   
99999  T037255          0          1          0    NaN    34    B    B    B   

        COL6  ...  COL42  COL43  COL44  COL47  COL48  COL49  COL50  COL51  \
0       0.00  ...   0.00   3.09    0.0   0.00    0.0    2.0  11.23  318.0   
1       0.00  ... -37.56  59.94    0.0   0.00    0.0    4.0  48.20    NaN   
2       0.00  ...  24.66  10.00    0.0   0.00    1.0    1.0  36.84   49.0   
3       0.00  ...   0.00  24.16    0.0  23.82    0.0    0.0   0.00   49.0   
4       0.00  ...   0.00  26.30    0.0   0.00    0.0    0.0   0.00   17.0   
...      ...  ...    ...    ...    ...    ...    ...    ...    ...    ...   
99995   0.00  ...   0.00   0.00    0.0   0.00    0.0    0.0   0.00    NaN   
99996   0.00  ...   0.00  20.88    1.0   0.00    1.0    0.0   0.00    4.0   
99997  60.78  ...   0.00  20.29    1.0  24.80    1.0    0.0   0.00   17.0   
99998   9.74  ...   0.00  28.36    1.0   0.00    1.0    0.0   0.00  292.0   
99999  58.43  ...  14.42  11.67    1.0  40.96    1.0    0.0   0.00   12.0   

       COL53  COL54  
0        0.0    0.0  
1        0.0    0.0  
2        0.0    0.0  
3        1.0    0.0  
4        0.0    0.0  
...      ...    ...  
99995    0.0    0.0  
99996    0.0    0.0  
99997    0.0    3.0  
99998    0.0    0.0  
99999    0.0    0.0  

[100000 rows x 49 columns]

In [4]:
Alpha = [chr(ord('A')+i) for i in range(26)]
IDs = [[i] for i in quiz['ID']]

In [5]:
class_mapping = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'H':8,'I':9,'J':10,'K':11,'L':12,'M':13,'N':14,'O':15,'P':16,'Q':17,'R':18,'S':19,'T':20,'U':21,'V':22,'W':23,'X':24,'Y':25,'Z':26}
names = ['COL3', 'COL4','COL5','COL19']
one_hot = []
# onehot
for name in names:
    if name in data.columns:
        temp_pd = pd.get_dummies(list(data[name])+Alpha,dtype=int)[:-26]
        temp_pd.columns = [i + name[-1] for i in list(temp_pd.columns)]
        data = pd.concat([data.drop(columns=[name]), temp_pd],axis=1)
        
        temp_pd = pd.get_dummies(list(quiz[name])+Alpha,dtype=int)[:-26]
        temp_pd.columns = [i + name[-1] for i in list(temp_pd.columns)]
        quiz = pd.concat([quiz.drop(columns=[name]), temp_pd],axis=1)
        one_hot.extend(temp_pd.columns)
# print(pd.get_dummies(list(data[names[3]])))
# print(enc.fit_transform(data[names[0]]))
# for name in names:
#     if name in data.columns:
#         data[name] = data[name].map(class_mapping)
#         quiz[name] = quiz[name].map(class_mapping)

for name in data.columns[1:]:
        data[name] = data[name].fillna(data[name].mean())
for name in quiz.columns[1:]:
        quiz[name] = quiz[name].fillna(quiz[name].mean())
data

ID  CHANNEL_A  CHANNEL_B  CHANNEL_C        COL1  COL2   COL6  \
0      T000689          0          0          0   28.102886    58   0.00   
1      T000848          0          1          0    6.000000    49   0.00   
2      T000967          0          0          0   28.102886    55   0.00   
3      T001249          0          0          0   17.000000    36   0.00   
4      T001524          0          1          0   22.000000    43   0.00   
...        ...        ...        ...        ...         ...   ...    ...   
99995  T034492          0          1          0   28.102886    30   0.00   
99996  T034682          0          0          0    4.000000    49   0.00   
99997  T035207          0          0          0   18.000000    30  60.78   
99998  T035311          0          0          0  347.000000    33   9.74   
99999  T037255          0          1          0   28.102886    34  58.43   

       COL7  COL8        COL9  ...  Q9  R9  S9  T9  U9  V9  W9  X9  Y9  Z9  
0         0     1   65.718368  ...   0   0   0   0   0   0   0   0   0   0  
1         0     1   24.000000  ...   0   0   0   0   0   0   0   0   0   0  
2         0     1   65.718368  ...   0   0   0   0   0   0   0   0   0   0  
3         0     0   41.000000  ...   0   0   0   0   0   0   0   0   0   0  
4         1     0   52.000000  ...   0   0   0   0   0   0   0   0   0   0  
...     ...   ...         ...  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
99995     0     0   65.718368  ...   0   0   0   0   0   0   0   0   0   0  
99996     0     0   29.000000  ...   0   0   0   0   0   0   0   0   0   0  
99997     0     0   40.000000  ...   0   0   0   0   0   0   0   0   0   0  
99998     2     1  377.000000  ...   0   0   0   0   0   0   0   0   0   0  
99999     2     0   65.718368  ...   0   0   0   0   0   0   0   0   0   0  

[100000 rows x 149 columns]

In [6]:
def get_adv_feats(df_train, df_test, feats):
    l_train = [1 for i in range(len(df_train))]
    adv_train = pd.DataFrame(l_train,columns=['adv'])
    df_train = pd.concat([df_train.reset_index(drop=True),adv_train],axis=1).reset_index(drop=True).sample(len(df_test),replace=True)
    l_test = [0 for i in range(len(df_test))]
    adv_test = pd.DataFrame(l_test,columns=['adv'])
    df_test = pd.concat([df_test.reset_index(drop=True),adv_test],axis=1).reset_index(drop=True)
    df = pd.concat([df_train, df_test]).reset_index(drop=True)
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'n_jobs': 4,
        'verbose': -1,
    }
    fold_num = 5
    steps = 3
    new_feats = []
    for f in feats:
        kf = StratifiedKFold(n_splits=fold_num, shuffle=True)
        for fold, (train_idx, val_idx) in enumerate(kf.split(df[[f]], df['adv'])):
            train = lgb.Dataset(df.loc[train_idx, [f]],
                                df.loc[train_idx, 'adv'])
            val = lgb.Dataset(df.loc[val_idx, [f]],
                              df.loc[val_idx, 'adv'])
            model = lgb.train(params, train, valid_sets=[val], num_boost_round=10000,  # feval=recall_score,
                              callbacks=[lgb.early_stopping(100), lgb.log_evaluation(-1)])
            x = model.predict(df.loc[val_idx, [f]])
            score = roc_auc_score(df.loc[val_idx, 'adv'], x)
            print(f,' --- ', score)
            if score > 0.8:
                new_feats.append(f)
                print('--------------------------------------', f, score)
            break
    return new_feats

# print(data.drop(columns=['CHANNEL_A','CHANNEL_B','CHANNEL_C']).columns == quiz.columns)
feats = quiz.drop(columns=['ID']).columns[:41]
del_feats = get_adv_feats(data.drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C']).sample(n=(10000),replace=True),
                      quiz.drop(columns=['ID']).sample(n=(10000),replace=True), feats)
print(del_feats)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[392]	valid_0's auc: 0.928108
COL1  ---  0.92810825
-------------------------------------- COL1 0.92810825
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11]	valid_0's auc: 0.594743
COL2  ---  0.5947431249999999
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[166]	valid_0's auc: 0.53785
COL6  ---  0.5378503750000001
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.519535
COL7  ---  0.5195350000000001
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5065
COL8  ---  0.5065
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[164]	valid_0's auc: 0.905693
COL9  ---  0.9056929999999999
-------------------------------

In [7]:
for drop_col in del_feats:
    if drop_col in data.columns:
        data = data.drop(columns=[drop_col])
    if drop_col in quiz.columns:
        quiz = quiz.drop(columns=[drop_col])

In [8]:
# resampling
Attributes = ['CHANNEL_A','CHANNEL_B','CHANNEL_C']
balance_data = {}

def resample(df, Attribute, length):
    pos_df = df[df[Attribute] == 1].sample(int(length/2),replace=True).reset_index(drop=True)
    neg_df = df[df[Attribute] == 0].sample(int(length/2),replace=True).reset_index(drop=True)
    new_df = pd.concat([pos_df,neg_df],axis=0).reset_index(drop=True)
    return new_df

In [9]:
# for Attribute in Attributes:
#     balance_data[Attribute] = resample(data, Attribute, 500000)

In [10]:
from sklearn.model_selection import train_test_split
dat_train = {}
dat_test = {}
for i in Attributes:
    dat_train[i], dat_test[i] = train_test_split(data, test_size=0.2, shuffle=True)
    dat_train[i] = resample(dat_train[i], i, 500000)

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
train_x = {}
train_y = {}
test_x = {}
test_y = {}
q = {}
scaler = StandardScaler()
# scaler = MinMaxScaler()

for i in Attributes:    
    train_x[i] = pd.DataFrame(scaler.fit_transform(dat_train[i].drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C']+one_hot)),
                           columns=dat_train[i].drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C']+one_hot).columns)
    train_x[i] = pd.concat([train_x[i],dat_train[i][one_hot].reset_index(drop=True)],axis=1)

    # print(train_x[i])

    train_y[i] = pd.DataFrame(dat_train[i][i].reset_index(drop=True))

    # print((train_y[i]))
    
    test_x[i] = pd.DataFrame(scaler.transform(dat_test[i].drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C']+one_hot)),
                           columns=dat_test[i].drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C']+one_hot).columns)
    test_x[i] = pd.concat([test_x[i],dat_test[i][one_hot].reset_index(drop=True)],axis=1)
    
    # print(test_x[i])
    
    test_y[i] = pd.DataFrame(dat_test[i][i].reset_index(drop=True))
    
    # print(test_y[i])

    q[i] = pd.DataFrame(scaler.transform(quiz.drop(columns=['ID']+one_hot)),
                           columns=quiz.drop(columns=['ID']+one_hot).columns)
    q[i] = pd.concat([q[i],quiz[one_hot].reset_index(drop=True)],axis=1)

In [12]:
best_params={'CHANNEL_A': {
   'booster': 'gbtree',
   'objective': 'binary:logistic', 
   'eval_metric': 'error', 
   'max_depth': 8, 
   'lambda': 2, 
   'gamma': 0, 
   'subsample': 0.75, 
   'colsample_bytree': 0.6, 
   'min_child_weight': 1,
   'eta': 0.015,
   'nthread': -1},
 'CHANNEL_B': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 7,
  'lambda': 2,
  'gamma': 0,
  'subsample': 0.75,
  'colsample_bytree': 0.75,
  'min_child_weight': 1,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 1.0},
 'CHANNEL_C': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 9,
  'lambda': 2,
  'gamma': 0,
  'subsample': 0.75,
  'colsample_bytree': 0.75,
  'min_child_weight': 1,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 1.0}}

In [14]:
F1_avg = 0
best_score = 0

ans = IDs

# eval_metrics = ['error','auc']
max_depths = [5,6,7,8,9,10,11,12,13,14,15]
lambdas = [2, 0, 1, 4]
gammas = [0, 0.5, 1]
subsamples = [0.75, 0.65, 0.85]
colsample_bytrees = [0.75, 0.6, 0.8]
min_child_weights = [1]
etas = [0.015, 0.05]

tot = len(max_depths)*len(lambdas)*len(gammas)*len(subsamples)\
        *len(colsample_bytrees)*len(min_child_weights)*len(etas)

F1_Sores = {'CHANNEL_A':0,'CHANNEL_B':0,'CHANNEL_C':0}

current_params = {
        'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'error',
        'max_depth':max_depths[0],
        'lambda':lambdas[0],
        'gamma':gammas[0],
        'subsample':subsamples[0],
        'colsample_bytree':colsample_bytrees[0],
        'min_child_weight':min_child_weights[0],
        'eta': etas[0],
        'nthread':-1
}
Params = {}
Final_score = {}
for i in Attributes:
    Params[i]=best_params[i]
    Final_score[i] = 0
# 'scale_pos_weight':pos_weight
tot

2376

In [15]:
Attributes = ['CHANNEL_A','CHANNEL_B','CHANNEL_C']
for i in Attributes:
    class_weights = {0: len(train_y[i])-train_y[i][i].sum(), 1: train_y[i][i].sum()}
    pos_weight = class_weights[0] / class_weights[1]
    print(pos_weight,end='  ')
    class_weights = {0: len(test_y[i])-test_y[i][i].sum(), 1: test_y[i][i].sum()}
    pos_weight = class_weights[0] / class_weights[1]
    print(pos_weight)

1.0  17.6219739292365
1.0  2.9385584875935407
1.0  108.2896174863388


In [16]:
def XGB_train(train_x,train_y,test_x,test_y,quiz,step,params):
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x)
    dquiz = xgb.DMatrix(quiz)
    class_weights = {0: len(train_y)-train_y[step].sum(), 1: train_y[step].sum()}
    pos_weight = class_weights[0] / class_weights[1]
    params['scale_pos_weight'] = pos_weight
    watchlist = [(dtrain,'train')]    
    
    model = xgb.train(params,dtrain,num_boost_round=200,evals=watchlist,obj=None,feval=None,maximize=False,early_stopping_rounds=100,
    evals_result=None,verbose_eval=False,xgb_model=None)

    ypred = model.predict( dtest )
    y_pred = (ypred >= 0.5) * 1
#     print(step)
#     print(y_pred.sum())
#     print(len(y_pred))
#     print('AUC: %.4f' % metrics.roc_auc_score(test_y, ypred))
#     print('ACC: %.4f' % metrics.accuracy_score(test_y, y_pred))
#     print('Precesion: %.4f' % metrics.precision_score(test_y, y_pred))
#     print('Recall: %.4f' % metrics.recall_score(test_y, y_pred))
#     print('F1-score: %.4f' % metrics.f1_score(test_y, y_pred))
#     print(metrics.confusion_matrix(test_y, y_pred))
#     print()
#     plot_importance(model)
    reply = (model.predict( dquiz ) >= 0.5) * 1
    for i in range(0,len(reply)):
        ans[i].append(reply[i])
    return metrics.f1_score(test_y, y_pred)

In [17]:
def Train(param,attribute):
    ans = IDs
    F1 = XGB_train(train_x[attribute],train_y[attribute],test_x[attribute],test_y[attribute],q[attribute],attribute, param)
    return F1

In [18]:
def tune(attribute):
    cnt = 0
    for eta in etas:
        for lambd in lambdas:
            for gamma in gammas:
                for subsample in subsamples:
                    for colsample_bytree in colsample_bytrees:
                        for min_child_weight in min_child_weights:
                            for max_depth in max_depths:
                                temp_params={'booster':'gbtree',
                                            'objective': 'binary:logistic',
                                            'eval_metric': 'error',
                                            'max_depth':max_depth,
                                            'lambda':lambd,
                                            'gamma':gamma,
                                            'subsample':subsample,
                                            'colsample_bytree':colsample_bytree,
                                            'min_child_weight':min_child_weight,
                                            'eta': eta,
                                            'nthread':-1}
                                current_params = temp_params
                                f1_s = Train(temp_params,attribute)
                                cnt += 1
                                print('\r',end='')
                                print(cnt/tot*100,'%',end='')
                                if f1_s > Final_score[attribute]:
                                    best_params[attribute] = temp_params
                                    Final_score[attribute] = f1_s
                                    print()
                                    print(f1_s)
                                    print(best_params[attribute])
                                    print("==========================")                                        

In [ ]:
tune(Attributes[1])

0.04208754208754208 %
0.44136819746575845
{'booster': 'gbtree', 'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 5, 'lambda': 2, 'gamma': 0, 'subsample': 0.75, 'colsample_bytree': 0.75, 'min_child_weight': 1, 'eta': 0.015, 'nthread': -1, 'scale_pos_weight': 1.0}
0.25252525252525254 %

In [481]:
f1x = 0
for i in Attributes:
    f1x += Train(best_params[i],i)
f1x/3.0

0.20216819527607102

In [373]:
best_score = max(best_score,F1_avg/3.0)
best_score

0

In [371]:
Params

{'CHANNEL_A': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 8,
  'lambda': 2,
  'gamma': 0,
  'subsample': 0.75,
  'colsample_bytree': 0.75,
  'min_child_weight': 2,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 1.0},
 'CHANNEL_B': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 7,
  'lambda': 2,
  'gamma': 0,
  'subsample': 0.75,
  'colsample_bytree': 0.75,
  'min_child_weight': 1,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 1.0},
 'CHANNEL_C': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 9,
  'lambda': 2,
  'gamma': 0,
  'subsample': 0.75,
  'colsample_bytree': 0.75,
  'min_child_weight': 1,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 1.0}}

In [372]:
F1_Sores

{'CHANNEL_A': 0.1654461621914836,
 'CHANNEL_B': 0.4861529199277544,
 'CHANNEL_C': 0.046224961479198766}

In [ ]:
sum(list(F1_Sores.values()))/3.0

In [ ]:
best_score

0.2319161220271427

In [ ]:
ans

In [ ]:
Output = pd.DataFrame(ans,columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C'],index=None)
Output

In [ ]:
Output.to_csv("output.csv",index=False)

{'CHANNEL_A': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 8,
  'lambda': 2,
  'gamma': 0.1,
  'subsample': 0.85,
  'colsample_bytree': 0.75,
  'min_child_weight': 1,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 110.42061281337047},
 'CHANNEL_B': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 8,
  'lambda': 2,
  'gamma': 1,
  'subsample': 0.85,
  'colsample_bytree': 0.75,
  'min_child_weight': 4,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 110.42061281337047},
 'CHANNEL_C': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 9,
  'lambda': 2,
  'gamma': 0.75,
  'subsample': 0.65,
  'colsample_bytree': 0.8,
  'min_child_weight': 4,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 110.42061281337047}}